## Redução de Dimensionalidade e Seleção de atributos

Foi visto anteriormente sobre a redução de dimensionalidade, onde algoritmos não-supervisionados (como o PCA) são capazes de condensar informações em um número menor de dimensão. No entanto, uma confusão que normalmente acontece com as terminologias, é a distinção entre selecionar atributos e reduzir dimensionalidade. Obviamente que a seleção de atributos acarretará em um número menor de dimensões, mas diferente da redução de dimensionalidade, a seleção de atributos descarta atributos que não são julgados como relevante para o problema, enquanto a redução condensa a informação em um espaço com menos dimensões.

---

Será utilizado o dataset digits conhecido no meio de aprendizado de máquina, importado diretamente do scikit-learn. Em seguida, serão aplicados PCA e seleção de atributos para analisar a diferença entre eles.

In [1]:
from sklearn.decomposition import PCA
from sklearn import datasets

digits = datasets.load_digits()
X = digits.data
y = digits.target

1

A função PCA importada de dentro do scikit-learn depende basicamente do atributo n_components. Esse atributo pode ser especificado como um inteiro, onde descreverá o número de componentes desejado, ou um valor contínuo entre 0 e 1, indicando o percentual de variância que deseja ser mantido no resultado final. O parâmetro whiten realiza uma operação matemática no sinal dos vetores extraídos no processo do PCA, e pode melhorar o resultado.

In [11]:
pca = PCA(n_components=0.99, whiten=True)

X_pca = pca.fit_transform(X)

print('Número original de atributos:', X.shape[1])
print('Número reduzido de atributos:', X_pca.shape[1])

Número original de atributos: 64
Número reduzido de atributos: 41


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Acurácia nos dados originais:', accuracy_score(y_test, y_pred))

#######

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.33, random_state=42)

model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Acurácia nos dados reduzidos:', accuracy_score(y_test, y_pred))

Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados reduzidos: 0.9747474747474747


### Seleção de atributos

A seleção de atributos não é um processo não-supervisionado, igual a redução de dimensionalidade. É feita uma análise diretamente relacionada com o rótulo da amostra, portanto é supervisionado. Uma das estratégias mais comuns de seleção de atributos é teste de valor ANOVA. Esse teste identifica se existe algum valor significamente diferente da relação entre atributo e rótulo. Os atributos que são muito diferentes nessa relação, podem ser descartados.

Lembrando sempre de não dar informações do teste para a seleção de atributos, para não virar um tipo de trapaça.

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#sc = StandardScaler()
#X_std = sc.fit_transform(X_train)

fvalue_selector = SelectKBest(f_classif, k=40)
X_kbest = fvalue_selector.fit_transform(X_train, y_train)

print('Número original de atributos:', X.shape[1])
print('Número reduzido de atributos:', X_kbest.shape[1])

model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Acurácia nos dados originais:', accuracy_score(y_test, y_pred))


model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)
model.fit(X_kbest, y_train)
X_test_kbest = fvalue_selector.transform(X_test)
y_pred = model.predict(X_test_kbest)
print('Acurácia nos dados kbest:', accuracy_score(y_test, y_pred))



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [ 0 32 39] are constant.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Número original de atributos: 64
Número reduzido de atributos: 40
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados kbest: 0.9646464646464646


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


A seleção de atributos preserva os atributos sem mudar o espaço original. Nesse caso, o resultado não parece ter sido favorável (97,3 dos dados originais vs 95,7 da seleção), enquanto a **redução de dimensionalidade proporcionou 97,4.** O resultado é ligeiramente melhor, mas o volume de dados processado é menor, então são ganhos reais de aplicação.

## Exercícios

1. Explore as configurações do PCA, utilizando números fixos de componentes e outros valores contínuos para a variância, e verifique se é possível obter melhores resultados utilizando a regressão logística e a separação de dados da forma como está.
2. Explore o número de atributos na função KBest, variando sistematicamente, para provar que há ou não há um resultado melhor.

In [61]:
import warnings
warnings.filterwarnings('ignore')
def testPCA(vVar):
    for i in vVar:
        print('N_componentes:',i)
        pca = PCA(n_components=i, whiten=True)
        X_pca = pca.fit_transform(X)
        print('Número original de atributos:', X.shape[1])
        print('Número reduzido de atributos:', X_pca.shape[1])
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print('Acurácia nos dados originais:', accuracy_score(y_test, y_pred))
        #######
        X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.33, random_state=42)
        model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print('Acurácia nos dados reduzidos:', accuracy_score(y_test, y_pred))
        print('---------------------------')
        


***Testando Variancia de 0.94 até 0.99***

In [47]:
var = [0.94,0.95,0.96,0.97,0.98,0.99]
testPCA(var)

N_componentes: 0.94
Número original de atributos: 64
Número reduzido de atributos: 27
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados reduzidos: 0.9612794612794613
---------------------------
N_componentes: 0.95
Número original de atributos: 64
Número reduzido de atributos: 29
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados reduzidos: 0.9629629629629629
---------------------------
N_componentes: 0.96
Número original de atributos: 64
Número reduzido de atributos: 31
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados reduzidos: 0.9663299663299664
---------------------------
N_componentes: 0.97
Número original de atributos: 64
Número reduzido de atributos: 34
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados reduzidos: 0.9696969696969697
---------------------------
N_componentes: 0.98
Número original de atributos: 64
Número reduzido de atributos: 37
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dado

In [50]:
var = [20,30,50,60]
testPCA(var)


N_componentes: 20
Número original de atributos: 64
Número reduzido de atributos: 20
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados reduzidos: 0.9528619528619529
---------------------------
N_componentes: 30
Número original de atributos: 64
Número reduzido de atributos: 30
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados reduzidos: 0.9629629629629629
---------------------------
N_componentes: 50
Número original de atributos: 64
Número reduzido de atributos: 50
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados reduzidos: 0.9612794612794613
---------------------------
N_componentes: 60
Número original de atributos: 64
Número reduzido de atributos: 60
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados reduzidos: 0.9646464646464646
---------------------------


In [63]:
def testSel(vVar):
    for i in vVar:
        print('kbest ',i)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

        #sc = StandardScaler()
        #X_std = sc.fit_transform(X_train)

        fvalue_selector = SelectKBest(f_classif, k=i)
        X_kbest = fvalue_selector.fit_transform(X_train, y_train)

        print('Número original de atributos:', X.shape[1])
        print('Número reduzido de atributos:', X_kbest.shape[1])

        model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print('Acurácia nos dados originais:', accuracy_score(y_test, y_pred))


        model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)
        model.fit(X_kbest, y_train)
        X_test_kbest = fvalue_selector.transform(X_test)
        y_pred = model.predict(X_test_kbest)
        print('Acurácia nos dados kbest:', accuracy_score(y_test, y_pred))
        print('--------------')

In [64]:
var = [1,15,30,40]
testSel(var)

kbest  1
Número original de atributos: 64
Número reduzido de atributos: 1
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados kbest: 0.23232323232323232
--------------
kbest  15
Número original de atributos: 64
Número reduzido de atributos: 15
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados kbest: 0.8888888888888888
--------------
kbest  30
Número original de atributos: 64
Número reduzido de atributos: 30
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados kbest: 0.9461279461279462
--------------
kbest  40
Número original de atributos: 64
Número reduzido de atributos: 40
Acurácia nos dados originais: 0.9730639730639731
Acurácia nos dados kbest: 0.9646464646464646
--------------
